<a href="https://colab.research.google.com/github/rshaikh95/sqlite_database_operations/blob/main/HospitalPricing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Loading Dataset
### Missing Values

In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('https://raw.githubusercontent.com/hantswilliams/HHA_504_2023/main/WK3/data/stonybrook/stonybrook.csv')


# Fill missing values
df['Description'].fillna('Unknown Description', inplace=True)
df['Type'].fillna('Inpatient', inplace=True)
df['Package/Line_Level'].fillna('Line', inplace=True)
df.fillna(0, inplace=True)




In [ ]:
import pandas as pd
import numpy as np
df1 = pd.read_csv('/content/1013998426_wsnchs-north-inc.-dba-st.-joseph-hospital_standardcharges (1).csv')

# Fill missing values
df1['Charge Description '].fillna('Unknown Description', inplace=True)
df1['Package Size'].fillna('Unknown', inplace=True)
df1['Package Unit'].fillna('Unknown', inplace=True)
df1['Package Description'].fillna('Unknown', inplace=True)
df1.fillna(0, inplace=True)
df1.replace()
df1.sample(50)

### Cleaning White Space and Characters

In [ ]:

import re

# Function to remove white space and special characters from a value
def clean_column_names(df):
    # Define a helper function to clean column names
    def clean_name(name):
        cleaned_name = re.sub(r'[^a-zA-Z0-9]', '', name)
        return cleaned_name.lower()

    # Rename columns using the helper function
    # This is using a list comprehend - e.g., we have a list to the right of the equals sign,
    # and inside the list, we are applying our function, for every col (or X) that exists in df.columns
    df.columns = [clean_name(col) for col in df.columns]
    return df

# Apply the clean_value function to all columns
df = clean_column_names(df)

df.columns




In [ ]:

import re

# Function to remove white space and special characters from a value
def clean_column_names(df1):
    # Define a helper function to clean column names
    def clean_name(name):
        cleaned_name = re.sub(r'[^a-zA-Z0-9]', '', name)
        return cleaned_name.lower()

    # Rename columns using the helper function
    # This is using a list comprehend - e.g., we have a list to the right of the equals sign,
    # and inside the list, we are applying our function, for every col (or X) that exists in df.columns
    df1.columns = [clean_name(col) for col in df1.columns]
    return df1

# Apply the clean_value function to all columns
df1 = clean_column_names(df1)

df1.columns




## Basic Statistics
Frequency Counts of Categories



In [ ]:
df['description'].value_counts()

In [ ]:
df['type'].value_counts()

In [ ]:
df['packagelinelevel'].value_counts()

In [ ]:
df1['chargedescription'].value_counts()

In [ ]:
df1['packageunit'].value_counts()

In [ ]:
df1['packagedescription'].value_counts()

Descriptive Statistics of Numerical Catagories

In [ ]:
# Generate descriptive statistics for numerical columns
print(df.describe())

In [ ]:
# Generate descriptive statistics for numerical columns

df1.fillna(0, inplace=True)
print(df1.describe(include = 'all'))
# Can't Figure out why NaN values appear when already replacing missing values earlier code


# SQLite Database Operations:


In [13]:
# Loading Packages
import pandas as pd
from sqlalchemy import create_engine
import sqlite3

In [14]:
# Creating a temporary + local DB using SQLITE
conn = sqlite3.connect('health.db')
c = conn.cursor()

In [15]:
c.execute("""
            CREATE TABLE stjoe
                (
                    codedescription text,
                    packagedescription text,
                    price real,
                    billingcode real
                );
          """)

conn.commit()

In [16]:
c.execute('''
  SELECT name
  FROM sqlite_master
  WHERE type='table';
  ''')

c.fetchall()

#for value in c.fetchall():
#    print(value)

[('stjoe',)]

In [17]:
c.execute('''
  SELECT * FROM stjoe;
''')

print(c.fetchall())

[]


## Inserting Fake Data


In [34]:
sql_query = """

INSERT INTO stjoe (
  'codedescription',
  'packagedescription',
  'price',
  'billingcode'
  )
  values (
    'XRX CLOZAPINE 25 MG PO TABS',
    'Blister',
    '8',
    '6370000002'
  );

"""

print(sql_query)



INSERT INTO stjoe (
  'codedescription',
  'packagedescription',
  'price',
  'billingcode'
  )
  values (
    'XRX CLOZAPINE 25 MG PO TABS',
    'Blister',
    '8',
    '6370000002'    
  );




In [35]:
c.execute(sql_query)
conn.commit()

In [36]:
sql_query_2 = """

select *
from stjoe;

"""

c.execute(sql_query_2)
print(c.fetchall())

[('XRX CLOZAPINE 25 MG PO TABS', 'Blister', 8.0, 6370000002.0)]


In [ ]:
# conn.close()

In [37]:
# ## create engine to connect to our sqlite DB
engine = create_engine('sqlite:///health.db')

In [ ]:
pd.read_sql_query("select * from stjoe;", conn)

## Load in some real data from StJoeseph - costs per CPT codes

In [ ]:
# Load in csv data from WK3/data/stonybrook/stonybrook.csv

dfr = pd.read_csv('/content/1013998426_wsnchs-north-inc.-dba-st.-joseph-hospital_standardcharges (1).csv')

# Shape
dfr.shape

# Columns
dfr.columns

## Create new column that goes from wide to long, where we keep columns 0:8 and then melt the rest into a new
## column called 'insurance_type'

columnNames = list(dfr)
idVars = columnNames[:8]
valueVars = columnNames[8:]

stjoe_modified = dfr.melt(id_vars=idVars, value_vars=valueVars)

stjoe_modified.columns

stjoe_modified.rename(columns={'codedescription':'codedescription',
            'packagedescription':'packagedescription',
            'billingcode':'billingcode',
            'price':'price'}, inplace=True)


stjoe_modified